In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle
import requests
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import optuna
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'images',
    validation_split=0.2,
    labels='inferred',
    label_mode='categorical',
    subset="training",
    color_mode='rgb',
    seed=123,
    image_size=(128, 128),
    batch_size=32)

# Get the shape of the data samples
sample_images, sample_labels = next(iter(train_ds))
sample_shape = sample_images.shape[1:]  # Excludes the batch dimension

print("Sample shape:", sample_shape)

Found 10824 files belonging to 2 classes.
Using 8660 files for training.
Sample shape: (128, 128, 3)


In [3]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    'images',
    validation_split=0.2,
    labels='inferred',
    label_mode='categorical',
    subset="validation",
    color_mode='rgb',
    seed=123,
    image_size=(128, 128),
    batch_size=32)

Found 10824 files belonging to 2 classes.
Using 2164 files for validation.


In [4]:
# Define the data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Randomly rotate the image by up to 20 degrees
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),  # Randomly flip the image horizontally
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),  # Randomly zoom the image by up to 20%
    tf.keras.layers.experimental.preprocessing.RandomTranslation(0.2, 0.2)  # Randomly shift the image horizontally and vertically by up to 20%
])

In [5]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [6]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.01,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

In [7]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=2,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

In [8]:
def objective(trial):
    model = Sequential([
        tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
        data_augmentation,
        Conv2D(trial.suggest_int('filters_1', 8, 12), (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Dropout(trial.suggest_float('dropout_1', 0.0, 0.5)),
        Conv2D(trial.suggest_int('filters_2', 16, 28), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(trial.suggest_float('dropout_2', 0.0, 0.5)),
        Conv2D(trial.suggest_int('filters_3', 32, 56), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(trial.suggest_float('dropout_3', 0.0, 0.5)),
        Conv2D(trial.suggest_int('filters_4', 64, 112), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(trial.suggest_float('dropout_4', 0.0, 0.5)),
        Flatten(),
        Dense(trial.suggest_int('units', 32, 64), activation='relu'),
        Dense(2, activation='softmax')
    ])

    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[early, reduce_lr, tb_callback])

    _, val_acc = model.evaluate(val_ds)
    return val_acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

[I 2023-06-26 19:11:31,277] A new study created in memory with name: no-name-44490f48-a400-478b-9ab9-981163e9bea6


Epoch 1/10


271/271 [==============================] - 59s 191ms/step - loss: 0.6352 - accuracy: 0.6145 - val_loss: 0.4503 - val_accuracy: 0.7837 - lr: 0.0010
Epoch 2/10
271/271 [==============================] - 47s 174ms/step - loss: 0.4786 - accuracy: 0.7718 - val_loss: 0.6344 - val_accuracy: 0.6811 - lr: 0.0010
Epoch 3/10
271/271 [==============================] - 49s 178ms/step - loss: 0.4010 - accuracy: 0.8230 - val_loss: 0.4399 - val_accuracy: 0.8101 - lr: 0.0010
Epoch 4/10
271/271 [==============================] - 49s 181ms/step - loss: 0.3533 - accuracy: 0.8472 - val_loss: 0.3303 - val_accuracy: 0.8697 - lr: 0.0010
Epoch 5/10
271/271 [==============================] - 49s 179ms/step - loss: 0.3314 - accuracy: 0.8569 - val_loss: 0.2811 - val_accuracy: 0.8900 - lr: 0.0010
Epoch 6/10
271/271 [==============================] - 49s 181ms/step - loss: 0.2994 - accuracy: 0.8727 - val_loss: 0.5340 - val_accuracy: 0.7791 - lr: 0.0010
Epoch 7/10
271/271 [==============================] - 52s 191ms

[I 2023-06-26 19:19:56,357] Trial 0 finished with value: 0.919593334197998 and parameters: {'filters_1': 11, 'dropout_1': 0.0008514694021172775, 'filters_2': 26, 'dropout_2': 0.40928189592635306, 'filters_3': 33, 'dropout_3': 0.05585563714761538, 'filters_4': 96, 'dropout_4': 0.09418337249852099, 'units': 38}. Best is trial 0 with value: 0.919593334197998.


Epoch 1/10
271/271 [==============================] - 55s 185ms/step - loss: 0.6679 - accuracy: 0.5836 - val_loss: 0.6106 - val_accuracy: 0.7158 - lr: 0.0010
Epoch 2/10
271/271 [==============================] - 51s 187ms/step - loss: 0.5539 - accuracy: 0.7163 - val_loss: 0.4839 - val_accuracy: 0.7676 - lr: 0.0010
Epoch 3/10
271/271 [==============================] - 52s 192ms/step - loss: 0.4372 - accuracy: 0.7923 - val_loss: 0.7393 - val_accuracy: 0.6354 - lr: 0.0010
Epoch 4/10
271/271 [==============================] - ETA: 0s - loss: 0.3768 - accuracy: 0.8334
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
271/271 [==============================] - 51s 189ms/step - loss: 0.3768 - accuracy: 0.8334 - val_loss: 0.5232 - val_accuracy: 0.7773 - lr: 0.0010
Epoch 5/10
271/271 [==============================] - 51s 188ms/step - loss: 0.3129 - accuracy: 0.8648 - val_loss: 0.3824 - val_accuracy: 0.8452 - lr: 1.0000e-04
Epoch 6/10
271/271 [========================

...

Epoch 1/10
271/271 [==============================] - 58s 193ms/step - loss: 0.6788 - accuracy: 0.5629 - val_loss: 0.5919 - val_accuracy: 0.7375 - lr: 0.0010
Epoch 2/10
271/271 [==============================] - 52s 193ms/step - loss: 0.5536 - accuracy: 0.7099 - val_loss: 0.4562 - val_accuracy: 0.7999 - lr: 0.0010
Epoch 3/10
271/271 [==============================] - 53s 193ms/step - loss: 0.4379 - accuracy: 0.7957 - val_loss: 0.5013 - val_accuracy: 0.7948 - lr: 0.0010
Epoch 4/10
271/271 [==============================] - 51s 187ms/step - loss: 0.3708 - accuracy: 0.8403 - val_loss: 0.3264 - val_accuracy: 0.8729 - lr: 0.0010
Epoch 5/10
271/271 [==============================] - 53s 193ms/step - loss: 0.3418 - accuracy: 0.8500 - val_loss: 0.3998 - val_accuracy: 0.8267 - lr: 0.0010
Epoch 6/10
271/271 [==============================] - 51s 187ms/step - loss: 0.3050 - accuracy: 0.8686 - val_loss: 0.2970 - val_accuracy: 0.8803 - lr: 0.0010
Epoch 7/10
271/271 [==============================] 

68/68 [==============================] - 4s 57ms/step - loss: 0.3476 - accuracy: 0.8526


[I 2023-06-26 19:45:25,729] Trial 3 finished with value: 0.8525878190994263 and parameters: {'filters_1': 12, 'dropout_1': 0.2335613932576922, 'filters_2': 27, 'dropout_2': 0.1692876976552556, 'filters_3': 55, 'dropout_3': 0.3510641182039921, 'filters_4': 86, 'dropout_4': 0.029810944261837558, 'units': 61}. Best is trial 0 with value: 0.919593334197998.


Epoch 1/10
271/271 [==============================] - 48s 159ms/step - loss: 0.6930 - accuracy: 0.5282 - val_loss: 0.6670 - val_accuracy: 0.6077 - lr: 0.0010
Epoch 2/10
271/271 [==============================] - 43s 158ms/step - loss: 0.5696 - accuracy: 0.7052 - val_loss: 0.4660 - val_accuracy: 0.8138 - lr: 0.0010
Epoch 3/10
271/271 [==============================] - 38s 141ms/step - loss: 0.5341 - accuracy: 0.7341 - val_loss: 0.3923 - val_accuracy: 0.8346 - lr: 0.0010
Epoch 4/10
271/271 [==============================] - 40s 148ms/step - loss: 0.4229 - accuracy: 0.8105 - val_loss: 0.3701 - val_accuracy: 0.8438 - lr: 0.0010
Epoch 5/10
271/271 [==============================] - 38s 138ms/step - loss: 0.3687 - accuracy: 0.8368 - val_loss: 0.3098 - val_accuracy: 0.8715 - lr: 0.0010
Epoch 6/10
271/271 [==============================] - 39s 144ms/step - loss: 0.3444 - accuracy: 0.8453 - val_loss: 0.2896 - val_accuracy: 0.8794 - lr: 0.0010
Epoch 7/10
271/271 [==============================] 

[I 2023-06-26 19:52:09,978] Trial 4 finished with value: 0.905730128288269 and parameters: {'filters_1': 10, 'dropout_1': 0.2542932450621956, 'filters_2': 23, 'dropout_2': 0.13574897111082496, 'filters_3': 49, 'dropout_3': 0.48832543310094906, 'filters_4': 78, 'dropout_4': 0.037218406027407835, 'units': 57}. Best is trial 0 with value: 0.919593334197998.


In [9]:
best_params = study.best_params

best_model = Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
    data_augmentation,
    Conv2D(best_params['filters_1'], (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Dropout(best_params['dropout_1']),
    Conv2D(best_params['filters_2'], (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(best_params['dropout_2']),
    Conv2D(best_params['filters_3'], (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(best_params['dropout_3']),
    Conv2D(best_params['filters_4'], (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(best_params['dropout_4']),
    Flatten(),
    Dense(best_params['units'], activation='relu'),
    Dense(2, activation='softmax')
])

best_model.compile(optimizer=Adam(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

best_model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[early, reduce_lr, tb_callback])

Epoch 1/50
271/271 [==============================] - 45s 148ms/step - loss: 0.6620 - accuracy: 0.5870 - val_loss: 0.5480 - val_accuracy: 0.7056 - lr: 0.0010
Epoch 2/50
271/271 [==============================] - 42s 154ms/step - loss: 0.4693 - accuracy: 0.7778 - val_loss: 0.4471 - val_accuracy: 0.7870 - lr: 0.0010
Epoch 3/50
271/271 [==============================] - 41s 152ms/step - loss: 0.3730 - accuracy: 0.8400 - val_loss: 0.4034 - val_accuracy: 0.8369 - lr: 0.0010
Epoch 4/50
271/271 [==============================] - 41s 149ms/step - loss: 0.3502 - accuracy: 0.8512 - val_loss: 0.3098 - val_accuracy: 0.8817 - lr: 0.0010
Epoch 5/50
271/271 [==============================] - 40s 147ms/step - loss: 0.3087 - accuracy: 0.8669 - val_loss: 0.3016 - val_accuracy: 0.8854 - lr: 0.0010
Epoch 6/50
271/271 [==============================] - 41s 150ms/step - loss: 0.2918 - accuracy: 0.8838 - val_loss: 0.2524 - val_accuracy: 0.9067 - lr: 0.0010
Epoch 7/50
271/271 [==============================] 

Epoch 12/50
271/271 [==============================] - 40s 147ms/step - loss: 0.2005 - accuracy: 0.9231 - val_loss: 0.1693 - val_accuracy: 0.9367 - lr: 0.0010
Epoch 13/50
271/271 [==============================] - ETA: 0s - loss: 0.2016 - accuracy: 0.9167
Epoch 13: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
271/271 [==============================] - 41s 151ms/step - loss: 0.2016 - accuracy: 0.9167 - val_loss: 0.1751 - val_accuracy: 0.9348 - lr: 0.0010
Epoch 14/50
271/271 [==============================] - 41s 149ms/step - loss: 0.1719 - accuracy: 0.9352 - val_loss: 0.1435 - val_accuracy: 0.9478 - lr: 1.0000e-04
Epoch 15/50
271/271 [==============================] - 42s 155ms/step - loss: 0.1623 - accuracy: 0.9355 - val_loss: 0.1463 - val_accuracy: 0.9459 - lr: 1.0000e-04
Epoch 16/50
271/271 [==============================] - ETA: 0s - loss: 0.1563 - accuracy: 0.9394
Epoch 16: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
271/271 [=========

In [10]:
def val_folder(dataset,folder):    
    l = []
    class_names = dataset.class_names
    for root, dirs, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(root, file)
            img = tf.keras.preprocessing.image.load_img(file_path, target_size=(128, 128, 3))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)
            predictions = best_model.predict(img_array)
            # Determine class label based on train dataset
            class_label = os.path.basename(root)
            if predictions[0][0] > 0.5:
                l.append(['small_table',class_label])
            else:
                l.append(['sofa',class_label])

    df = pd.DataFrame(l).rename(columns={0:'Predictions', 1:'Actuals'})

    df['check'] = np.where(df['Predictions']!=df['Actuals'],0,1)

    x = df.groupby('Predictions',as_index=False).sum()['check']

    y = df.groupby('Predictions',as_index=False).count()['check']

    print(f"{class_names[0]} has {x[0]} correct predictions and {y[0]-x[0]} incorrect with a {round(x[0]/y[0],2)} accuracy.")
    print(f"{class_names[1]} has {x[1]} correct predictions and {y[1]-x[1]} incorrect with a {round(x[1]/y[1],2)} accuracy.")

In [11]:
val_folder(train_ds,'test')

1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 16ms/step
small_table has 511 correct predictions and 25 incorrect with a 0.95 accuracy.
sofa has 545 correct predictions and 14 incorrect with a 0.97 accuracy.


In [13]:
best_model.save("furn.h5")
best_model.save_weights('furn_weights.h5')

## Extract feature vectors

### Load model up to classification layer

In [15]:
loaded_model = load_model("furn.h5",compile=False)
layers_to_load = loaded_model.layers[:-3]
new_model = tf.keras.models.Sequential(layers_to_load)
new_model.build((None,128,128,3))
new_model.load_weights('furn_weights.h5', by_name=True)

### Extract vectors

In [16]:
f_vectors = []
for i in train_ds: 
    predictions = new_model.predict(i)
    f_vectors.append(predictions)
    
all_vecs = np.concatenate(f_vectors, axis=0)

1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 67ms/step


In [17]:
np.save('all_vecs.npy',all_vecs)

In [18]:
all_vecs = np.load('all_vecs.npy')
all_vecs = all_vecs.reshape(len(all_vecs),-1)